In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
import seaborn as sns  # This optional package makes plots prettier

import openmc

## Inspecting HDF5 Pointwise Data

OpenMC does not use ACE cross sections.  It uses a purpose-built HDF5 data format.  The PythonAPI has full support for reading and modifying this format.  A Python object can be generated from an HDF5 file like this:

In [ ]:
library = openmc.data.DataLibrary.from_xml(os.environ['OPENMC_CROSS_SECTIONS'])
filename = library.get_by_material('U238')['path']
u238_pointwise = openmc.data.IncidentNeutron.from_hdf5(filename)

Cross sections are accessed through the reaction "MT" number

In [ ]:
for r in u238_pointwise.reactions.values(): print(r)

In [ ]:
n_gamma = u238_pointwise[102]
n_gamma.xs

One HDF5 file can contain cross sections at many temperatures.  OpenMC can use this to interpolate between temperatures.  In this case, there is only one temperature.  This example shows how to plot every point in the dataset.

In [ ]:
xs = n_gamma.xs['294K']
plt.loglog(xs.x, xs.y)

This example shows how to evaluate a cross section at arbitrary energy (with linear interpolation).

In [ ]:
E = np.linspace(5, 25, 1000)
plt.semilogy(E, xs(E))

The PythonAPI can read ACE data with the method `openmc.data.IncidentNeutron.from_ace(...)`.  It can then output an HDF5 file with the `.export_to_hdf5(...)` method.  The PythonAPI even allows you to modify the data before export.

## Windowed Multipole Data

OpenMC can also be used with an experimental format called windowed multipole.

In [ ]:
filename = os.environ['OPENMC_MULTIPOLE_LIBRARY'] + '/092238.h5'
u238_multipole = openmc.data.WindowedMultipole.from_hdf5(filename)

Windowed multipole allows for analytic Doppler broadening.  The Python object can generate cross sections for any reasonable temperature and any energy in the resolved resonance range.  Calling a `WindowedMultipole` object gives 3 cross sections: total, (n, gamma), and fission

In [ ]:
u238_multipole(1.0, 294)

Here is how to make the same plot of (n, gamma) cross section as before.

In [ ]:
E = np.linspace(5, 25, 1000)
plt.semilogy(E, u238_multipole(E, 293.606)[1])

This plot shows the Doppler broadening between 0 K and 900 K.

In [ ]:
E = np.linspace(6.1, 7.1, 1000)
plt.semilogy(E, u238_multipole(E, 0)[1])
plt.semilogy(E, u238_multipole(E, 900)[1])